In [1]:
import sys
sys.path.append('/ssd/kangdang/mmc_project_2023_submission/main_exp_code/step4_exp_analysis')

import pickle
import numpy as np
import os
import os.path as pathlib
from stats_fun import get_score_and_gt, get_statistic_info, cal_ci95
import matplotlib.pyplot as plt
base_path = '/mnt/eye_team/kangdang/mmc_project_2023_submission/exp_records/fusion_exps/random_mi_single_variable/'

# image only result
test_result_pickle = pickle.load(open('/mnt/eye_team/kangdang/mmc_project_2023_submission/exp_records/image_only_exps/top_checkpoint_ensemble_test_result.pickle', 'rb'))
test_pred_set = test_result_pickle['pred_set']
test_gt_set = test_result_pickle['gt_set']
auc95_dl, fpr_dl, tpr_dl, auc_dl = get_statistic_info(test_gt_set, test_pred_set)
print(auc95_dl)
auc95, rec, sep, sep_90, global_auc, global_rec, global_sep, global_sep_90, _, _ = cal_ci95(test_gt_set, test_pred_set, 0.8)
print(rec)
print(global_rec)
print(sep)
print(global_sep)

[0.7705749921531228, 0.7898181694319033]
[0.6677000581282697, 0.6889540566959922]
0.6779759596742924
[0.7309417040358744, 0.7617967332123412]
0.7459733087896916


In [2]:
def get_indicator_all_results():
    indicator_id_list = ["'1'", "'2'", "'3'", "'4'", "'5'", "'6'", "'7'", "'8'", "'9'", "'10'", "'11'",  "'12'"]
    indicator_name_list = ['Gender', 'Age', 'Diastolic Bp', 'Systolic Bp',  'Heart Rate', 'BMI', 'Course of Diabetes', 'Hypertension', 'Hyperlipidemia', 'Cardiovascular Disease', 'Classification of smoker', 'Classification of drinker']
    result_dict = dict()
    for indicator_id, indicator_name in zip(indicator_id_list, indicator_name_list):
        print(indicator_name)
        best_method_id, train_gt_list, val_gt_list, test_gt_list, train_pred_list, val_pred_list, test_pred_list = get_score_and_gt(base_path, indicator_id, method_id_list = ['method_lr_basic'])
        auc95, fpr, tpr, auc = get_statistic_info(test_gt_list, test_pred_list)
        result_dict[indicator_name] = [best_method_id, auc95, fpr, tpr, auc]
    return result_dict

result_dict = get_indicator_all_results()
sort_dict = dict()
for key, value in result_dict.items():
    sort_dict[key] = -value[-1]
sort_keys = sorted(sort_dict, key=sort_dict.get)

Gender
Age
Diastolic Bp
Systolic Bp
Heart Rate
BMI
Course of Diabetes
Hypertension
Hyperlipidemia
Cardiovascular Disease
Classification of smoker
Classification of drinker


In [ ]:
import matplotlib as mpl
my_dpi=120
plt.figure(figsize=(960/my_dpi, 960/my_dpi), dpi=my_dpi)
plt.style.use('seaborn-darkgrid')
palette = plt.get_cmap('Set3')
plt.plot(fpr_dl, tpr_dl, marker='', color='red', linewidth=6, alpha=0.7, label='Image only\nAUC='+ "{:.3f}".format(auc_dl)+ '[{:.3f}'.format(auc95_dl[0]) + ',{:.3f}'.format(auc95_dl[1]) + ']')
cmap = mpl.cm.get_cmap('PuOr')
prespace = [12, 3, 0, 3, 3, 0, 0, 3, 6, 3, 6, 0]
for idx, key in enumerate(sort_keys):
    value = result_dict[key]
    fpr = value[2]
    tpr = value[3]
    label = key + '\nAUC= ' + "{:.2f}".format(value[4]) + '[{:.3f}'.format(value[1][0]) + ',{:.3f}'.format(value[1][1]) + ']'
    color = cmap((idx/len(sort_keys)))
    plt.plot(fpr,tpr, marker='', color=color, linewidth=3, alpha=0.5, label=label)
plt.legend(prop={'size': 14})

ax=plt.gca()
ax.xaxis.label.set_fontsize(15)
ax.yaxis.label.set_fontsize(15)

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Test Set")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5),labelspacing=1, prop={'size': 12})
